Load the environment variables

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

Marking the GraphState

In [2]:
from typing import TypedDict, Annotated, List, Union, Sequence
from langchain_core.agents import AgentAction, AgentFinish
from langchain_core.messages import BaseMessage
import operator

class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]

Custom Tools

Tools are interfaces that an agent can use to interact with the world. They combine a few things:
1. The name of the tool
2. A description of what the tool is
3. JSON schema of what the inputs to the tool are
4. The function to call
5. Whether the result of a tool should be returned directly to the user

In [3]:
import random
from langchain_community.tools.tavily_search import TavilySearchResults

tavily_tool = TavilySearchResults(max_results=5)

tools = [tavily_tool]


Agent - with binding

In [4]:
from langchain import hub
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.utils.function_calling import convert_to_openai_function

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")

# Choose the LLM that will drive the agent
model = ChatOpenAI( temperature=0, streaming=True)

# Bind the functions to model
functions = [convert_to_openai_function(t) for t in tools]
model = model.bind_functions(functions)

Nodes

In [5]:
# Tools Executor
from langgraph.prebuilt.tool_executor import ToolExecutor

tool_executor = ToolExecutor(tools)

In [6]:
from langchain_core.agents import AgentFinish
from langgraph.prebuilt import ToolInvocation
import json
from langchain_core.messages import FunctionMessage

# Define logic that will be used to determine which conditional edge to go down
def should_continue(state):
    messages = state['messages']
    last_message = messages[-1]
    # If there is no function call, then we finish
    if "function_call" not in last_message.additional_kwargs:
        return "end"
    # Otherwise we continue
    else:
        return "continue"

# Define the function that calls the model
def call_model(state):
    messages = state['messages']
    response = model.invoke(messages)
    # We return a list, because this will get added to the existing list
    return {"messages": [response]}

# Defin the function to execute tools
def call_tool(state):
    messages = state['messages']
    # Based on the continue condition
    # we know the last message involves a function call
    last_message = messages[-1]
    # We construct a ToolInvocation from the function_call
    action = ToolInvocation(
        tool=last_message.additional_kwargs["function_call"]["name"],
        tool_input=json.loads(last_message.additional_kwargs["function_call"]["arguments"]),
    )
    print(f"The agent action is {action}")
    # We call the tool_executor and get back a response
    response = tool_executor.invoke(action)
    print(f"The tool result is: {response}")
    # We use the response to create a FunctionMessage
    function_message = FunctionMessage(content=str(response), name = action.tool)
    # We return a list, because this will get added to the existing list
    return {"messages": [function_message]}


Define the graph

In [7]:
from langgraph.graph import END, StateGraph

# Define a new graph
workflow = StateGraph(AgentState)

# Define the two nodes we will cycle between
workflow.add_node("agent", call_model)
workflow.add_node("action", call_tool)

# Set the entrypoint as `agent`
# This means that this node is the first one called
workflow.set_entry_point("agent")

# We now add a conditional edge
workflow.add_conditional_edges(
    # First, we define the start node. We use `agent`.
    # This means these are the edges taken after the `agent` node is called.
    "agent",
    # Next, we pass in the function that will determine which node is called next.
    should_continue,
    # Finally, we pass in a mapping.
    # The keys are strings, and the values are other nodes.
    # END is a special node marking that the graph should finish.
    # What will happen is we will call `should_continue`, and then the output of that
    # will be matched against the keys in this mapping.
    # Based on which one it matches, that node will then be called.
    {
        # If `tools`, then we call the tool node.
        "continue": "action",
        # Otherwise we finish
        "end": END
    }
)

# We now add a normal edge from `tools` to `agent`.
# This means that adter `tools` is called, `agent` node is called next.
workflow.add_edge('action', 'agent')

# Finally, we compile it!
# This compiles it into a LangChain Runnable,
# meaning you can use it as you would any other runnable
app = workflow.compile()


Run it!!

In [8]:
from langchain_core.messages import HumanMessage, SystemMessage

system_message = SystemMessage(content="You are a very helpful assistant")
user_01 = HumanMessage(
    content=
    "Please fetch publicly available information about Udupi Sandeep Shenoy Zeta."
    " Then, Summarise this information in 2 paragraphs of not more than 3 sentences."
    " Also, provide all available social media URLs of that person"
)
inputs = {
    "messages": [system_message, user_01]
}

app.invoke(inputs)

The agent action is tool='tavily_search_results_json' tool_input={'query': 'Udupi Sandeep Shenoy Zeta'}
The tool result is: [{'url': 'http://mahalaxmieyeclinic.com/about_doctors.html', 'content': 'Doctors. DR SANDEEP SHENAI.K : Did his MBBS from Kasturba Medical College Manipal and then M.S (Ophthal) from the same college. Was recepient of Dr Srinivas Rao best out going student in1994. Worked under Dr Suhas Haldipurkar at Laxmi Eye Institute,Panvel before establishing Mahalaxmi Eye Clinic and Laser Centre in 1996.'}, {'url': 'https://www.zoominfo.com/p/Udupi-Shenoy/10157168616', 'content': "Get the details of Udupi Shenoy's business profile including email address, phone number, work history and more. Products. ... Manager, Engineering at Zeta India Incorporation View Contact Info for Free . Udupi Shenoy Email & Phone number. Engage via Email. u***@zetaindia.net."}, {'url': 'https://www.linkedin.com/pub/dir/Sandeep/Shenoy/in-0-India', 'content': 'There are 90+ professionals named "Sand

{'messages': [SystemMessage(content='You are a very helpful assistant'),
  HumanMessage(content='Please fetch publicly available information about Udupi Sandeep Shenoy Zeta. Then, Summarise this information in 2 paragraphs of not more than 3 sentences. Also, provide all available social media URLs of that person'),
  AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"query":"Udupi Sandeep Shenoy Zeta"}', 'name': 'tavily_search_results_json'}}, response_metadata={'finish_reason': 'function_call'}, id='run-8631ad2b-4fc6-447e-a3e5-9a6b96e31207-0'),
  FunctionMessage(content='[{\'url\': \'http://mahalaxmieyeclinic.com/about_doctors.html\', \'content\': \'Doctors. DR SANDEEP SHENAI.K : Did his MBBS from Kasturba Medical College Manipal and then M.S (Ophthal) from the same college. Was recepient of Dr Srinivas Rao best out going student in1994. Worked under Dr Suhas Haldipurkar at Laxmi Eye Institute,Panvel before establishing Mahalaxmi Eye Clinic and Laser Centre in 